In [1]:
import sshtunnel
from flask import Flask
from flask_migrate import Migrate
from flask_wtf.csrf import CSRFProtect
from flask_sqlalchemy import SQLAlchemy
from dbenv import SSH_USERNAME,SSH_PASSWORD,DB_NAME,DB_PASSWORD,HOST,PORT,REMOTE_BIND_ADRESS
from datetime import datetime
from flask_bootstrap import Bootstrap

#pythonanywhere mysql bağlantısı
tunnel = sshtunnel.SSHTunnelForwarder(('ssh.pythonanywhere.com'),
                                      ssh_username=SSH_USERNAME,
                                      ssh_password=SSH_PASSWORD,
                                      remote_bind_address=(REMOTE_BIND_ADRESS,PORT)
                                      )
tunnel.start()
app = Flask(__name__)

@app.context_processor
def today():
    return {'now': datetime.now().strftime("%Y-%m-%d")}


app.config['SECRET_KEY'] = "KFAŞLKDJAFIDSFcnzndklsfjsdfjs"
app.config['SQLALCHEMY_DATABASE_URI'] = f'mysql://{SSH_USERNAME}:{DB_PASSWORD}@{HOST}:{tunnel.local_bind_port}/{DB_NAME}'

csrf = CSRFProtect(app)
db = SQLAlchemy(app)
Bootstrap(app)

migrate = Migrate(app,db)

In [17]:
from mod_forecast.model import getBydate,getnotnulldata,getnulldata
import pandas as pd
import numpy as np

In [18]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [52]:
#@app.route('/traindate/<date>,<hours>',methods=['POST','GET'])

import json

def traindate( date , hours ):
    if getBydate(str(pd.to_datetime(date).date())):
        actual=[row.GercekDeger for row in getBydate(str(pd.to_datetime(date).date()))[:int(hours)]]
        predicted = [row.TahminiDeger for row in getBydate(str(pd.to_datetime(date).date()))[:int(hours)]]

        for_mape_actual = [x for x in actual if x not in [None]]
        for_mape_predicted = predicted[:len([x for x in actual if x not in [None]])]
        now_mape=mean_absolute_percentage_error(for_mape_actual,for_mape_predicted)
        return now_mape, actual,predicted


#@app.route('/')
#def index():
#    return "Hello World"
#
#if __name__ == '__main__':
#    app.run(port=5998)



In [2]:
from flask import Blueprint,request,jsonify,abort
from flask_jwt_extended import jwt_required,get_jwt_identity
import numpy as np
import pandas as pd
mod_deeplearningV1 = Blueprint('customer',__name__)
mod_deeplearningV2 = Blueprint('developer',__name__)


In [51]:

@mod_deeplearningV2.route('/calculatebenefit',methods=["POST"])
def dashboardCalculateBenefit_hit():
    from mod_forecast_hit.model import allForecastBySFId, allForecastBydev
    if request.method == "POST":
        req_jeson = request.json
        companyId = req_jeson['companyId']
        from mod_storeandforward_hit.model import getModelsByCompany
        if len(getModelsByCompany(companyId)) == 0:
            return jsonify(
                message="There is no entry submitted by user. Select specify entry to be processed."), 409  # ?
        else:
            sf_id = []
            for i in getModelsByCompany(companyId):
                sf_id.append(i.Id)
            allForecastByCompany = {"Datetime": [],
                                    "Actual": [],
                                    "Prediction": []}
            for i in allForecastBydev(sf_id):
                allForecastByCompany["Datetime"].append(i.Tarih)
                allForecastByCompany["Actual"].append(i.GercekDeger)
                allForecastByCompany["Prediction"].append(i.TahminiDeger)
            return jsonify(
                {'Data': {'Date': allForecastByCompany["Datetime"],
                          'Actual': allForecastByCompany["Actual"],
                          'Predicted': allForecastByCompany['Prediction']}})
